In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [ ]:
data = datasets.load_breast_cancer()
data

In [ ]:
X = data.data
y = data.target

In [ ]:
# data_type -> float32로 변환
X = np.array(X, dtype='float32')
y = np.array(y, dtype='float32')

In [ ]:
y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)

In [ ]:
def sigmoid(n) :
	return 1 / (1 + np.exp(-n))

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

w = tf.Variable(initializer([30, 1]))

In [ ]:
w

In [ ]:
b = tf.Variable(initializer([1]))
b

In [ ]:
tf.matmul(X_train, w) + b

In [ ]:
# 로지스틱 회귀식
hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)
hypothesis

In [ ]:
# w, b를 갱신할 객체생성
# tf.train.AdamOptimizer(learning_rate)
optimizer = tf.train.AdamOptimizer(0.001)

In [ ]:
# 반복해서 w, b 값을 찾는 방법
for step in range(100) :
	with tf.GradientTape() as tape :
		# 로지스틱 회귀식
		hypothesis = tf.sigmoid(tf.matmul(X_train, w) + b)

		# 로지스틱 회귀식의 오차 계산식
		cost = -tf.reduce_mean(y_train * tf.log(hypothesis) + (1 - y_train) * tf.log(1 - hypothesis))

		# cost, w, b를 이용해 w와 b의 기울기 계산
		grads = tape.gradient(cost, [w, b])

	# 최적의 w, b 값을 찾기 위해 w와 b를 갱신
	optimizer.apply_gradients(grads_and_vars=zip(grads, [w, b]))

	if step % 10 == 0 :
		print('=' * 50)
		print(f'step = {step}, cost = {cost}, w = {w}, b = {b}')
		print('=' * 50)
		

In [ ]:
w

In [ ]:
b

In [ ]:
# X_test에서 X_train의 평균을 빼고 X_train의 표준편차로 나눠줌
X_test = scaler.transform(X_test)
X_test

In [ ]:
# X_test에 w를 곱하고 b를 더하고 활성함수 sigmoid를 이용해 데이터 범위를 0 ~ 1로 설정
predict = tf.sigmoid(tf.matmul(X_test, w) + b)
predict

In [ ]:
predict01 = tf.cast(predict > 0.5, dtype=tf.float32)
print('=' * 40)
print('predict01')
print(predict01)

In [ ]:
# 조건을 만족하면 해당 줄칸에 True를 반환 다르면 False를 반환
ac01 = tf.equal(predict01, y_test)
ac01

In [ ]:
# tf.cast() -> 조건을 만족하지 못하면 False 만족하면 True
# 			   True는 1로 False는 0으로 반환
ac02 = tf.cast(ac01, dtype='float32')
ac02

In [ ]:
ac03 = tf.reduce_mean(ac02)
ac03

In [ ]:
# 정확도를 계산
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict01, y_test), dtype=tf.float32))
print('=' * 40)
print('accuracy')
print(accuracy.numpy())